In [26]:
# import dependencies
import os
import matplotlib as plt
import pandas as pd
import numpy as np
import time
import datetime
import json
import requests as req
from citipy import citipy

In [27]:
# get dataset
# TODO: remove warning
dataset_path = os.path.join('Data', 'flights.csv')
dataset = pd.read_csv(dataset_path)
airport_data = json.load(open('Data/airports.json', 'r'))

C:\Users\joeba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
# convert to dataframe
flight_df = pd.DataFrame(dataset)
#LAX,Chicago,New York, Atlanta, Dallas, JFK, San Fran, Las Vegas, Charlotte, Miami
#LAX, 

In [29]:
# add delayed column to show if flight was delayed (1) or not (0)
flight_df['Delayed'] = ''
# add column for the date
flight_df['Date'] = ''
# create column for unix timestamp
flight_df['Unix Time'] = ''
# columns for lat and lon
flight_df['origin_lat'] = ''
flight_df['origin_lon'] = ''
flight_df['origin_city'] = ''
flight_df['dest_lat'] = ''
flight_df['dest_lon'] = ''
flight_df['dest_city'] = ''

In [30]:
# set Delayed column if flight was delayed
for index, row in flight_df.head(30).iterrows():
    # TODO: check if other columns are also null
    if(pd.isnull(row['AIR_SYSTEM_DELAY']) == False):
        flight_df.set_value(index, 'Delayed', 1)
    
    date = str(row['MONTH']) + '/' + str(row['DAY']) + '/' + str(row['YEAR'])
    
    flight_df.set_value(index, 'Date', date)

In [31]:
for index, row in flight_df.head(30).iterrows():
    unix_time = time.mktime(datetime.datetime.strptime(row['Date'], "%m/%d/%Y").timetuple())
    
    flight_df.set_value(index, 'Unix Time', int(unix_time))

In [44]:
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,WEATHER_DELAY,Delayed,Date,Unix Time,origin_lat,origin_lon,origin_city,dest_lat,dest_lon,dest_city
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,,1/1/2015,1420092000,,,,,,
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,,1/1/2015,1420092000,,,,,,
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,,1/1/2015,1420092000,,,,,,
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,,1/1/2015,1420092000,,,,,,
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,,1/1/2015,1420092000,,,,,,


In [33]:
print(flight_df['Unix Time'][0])

1420092000


In [34]:
# get average stats for each airline
avg_airline_stats_df = flight_df.groupby('AIRLINE').mean()

In [35]:
# remove unecessary columns that don't need averaged
avg_airline_stats_df.drop(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)

In [36]:
# sort by the average arrival delay from longest avg delay to shortest avg delay
avg_airline_stats_df.sort_values('ARRIVAL_DELAY', ascending=False)

,DEPARTURE_DELAY,TAXI_OUT,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
AIRLINE,,,,,,,,,,,,
NK,15.944766,14.604656,156.503598,132.347886,985.267118,9.556557,14.471800,27.510096,0.150405,13.766839,20.490868,1.288332
F9,13.350858,15.674198,155.056677,130.240571,967.214860,9.147244,12.504706,24.659907,0.000000,14.719983,26.900255,0.917098
B6,11.514353,17.872453,167.838858,143.858492,1062.175178,6.121373,6.677861,16.754728,0.192936,18.150503,23.954305,1.956401
EV,8.715934,16.736971,97.124247,72.763842,462.251739,7.635511,6.585379,15.459170,0.000000,21.651277,24.078400,1.550713
MQ,10.125188,16.549323,92.593516,66.979630,422.315407,9.084096,6.457873,15.372752,0.116422,17.425025,23.404512,6.644508
OO,7.801104,18.146070,100.724558,75.655010,496.772164,6.933260,5.845652,12.375082,0.091804,18.959163,26.612403,2.322232
UA,14.435441,17.414003,190.745816,164.835886,1271.545684,8.508092,5.431594,14.422681,0.012462,21.144678,26.019098,3.103875
VX,9.022595,14.755854,205.558435,182.630796,1405.989322,8.186914,4.737706,20.360672,0.155035,11.097809,21.693751,2.994057
WN,10.581986,11.943682,121.201368,103.100485,740.711335,6.167662,4.374964,7.054161,0.050240,16.191674,26.681590,2.304772


In [37]:
# get average stats for each month
avg_month_stats_df = flight_df.groupby('MONTH').mean()
avg_month_stats_df.drop(['YEAR', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)
avg_month_stats_df.sort_values('ARRIVAL_DELAY', ascending=False)

,DEPARTURE_DELAY,TAXI_OUT,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
MONTH,,,,,,,,,,,,
6,13.986016,16.286506,138.385980,114.487011,835.630272,7.629777,9.601590,14.090546,0.061205,19.840101,26.402136,3.289204
2,11.885284,16.918242,137.079924,112.593760,800.785450,7.583330,8.320500,14.175112,0.048120,17.994095,22.672207,4.320964
7,11.394664,16.209543,138.148580,114.426570,841.477279,7.524097,6.431775,12.482509,0.063376,18.781337,25.852072,2.159941
12,11.783529,16.212791,139.862108,116.160356,837.801893,7.504642,6.092903,13.304096,0.104390,19.593787,26.236122,3.378023
1,9.759471,16.456498,136.491717,112.544096,803.261279,7.496929,5.813583,13.319872,0.069827,17.802368,22.760211,2.741889
3,9.660903,15.893529,136.677481,113.468007,816.055327,7.328933,4.920673,12.874460,0.068998,19.053346,22.588212,2.395560
8,9.932045,16.086252,137.167441,113.566323,834.824428,7.527608,4.607372,13.543326,0.127432,19.334183,23.848831,2.824179
5,9.454144,15.712685,136.993199,114.012205,823.323059,7.281282,4.485019,13.995783,0.063707,18.605834,24.234715,3.754309
4,7.721885,15.707033,136.778517,113.918162,817.006048,7.165978,3.163190,13.593529,0.044500,18.123384,21.708233,2.694202


In [38]:
# get average stats for each month
avg_month_stats_df = flight_df.groupby('DAY_OF_WEEK').mean()
avg_month_stats_df.drop(['YEAR', 'DAY', 'MONTH', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)
avg_month_stats_df.sort_values('ARRIVAL_DELAY', ascending=False)

,DEPARTURE_DELAY,TAXI_OUT,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
DAY_OF_WEEK,,,,,,,,,,,,
1,10.871775,16.178975,136.558717,112.901300,816.347052,7.496895,6.002063,14.639036,0.077929,19.034018,25.509040,3.484585
4,9.956913,16.555765,137.335089,113.352508,818.616981,7.438442,5.668334,14.320178,0.063358,17.608450,23.126220,2.774602
5,9.429768,16.221626,136.702400,112.998546,817.928374,7.492879,4.764079,12.610116,0.057348,18.297777,22.901769,2.592428
2,9.164522,16.079230,135.502257,112.051591,809.096095,7.379613,4.246041,14.088773,0.074307,18.710318,24.052387,3.089964
7,9.402856,15.770759,137.621344,114.365818,831.991691,7.498137,3.958293,12.736719,0.086084,20.109441,23.643850,2.758606
3,8.645277,16.232548,135.961037,112.360630,811.784365,7.374403,3.845468,13.066541,0.079314,18.331489,23.111006,2.462534
6,7.827416,15.298064,139.884260,117.252399,857.545201,7.350544,1.852892,12.451199,0.106160,21.711426,21.271223,3.354304


In [88]:
# TODO: loop through all rows and get lat/lon
# TODO: use this info to get the weather for that airport on that day
# USE JSON TO GET CITY NAME, DONT NEED LAT & LON
#for index, row in flight_df.iterrows():
#    for key in airport_data:
#        if key['code'] == row['ORIGIN_AIRPORT']:
#            #city.append(key['city'])
#            flight_df.loc[index, 'origin_city'] = key['city']
#            break


In [101]:
airport_city_code = {}
for key in airport_data:
    airport_city_code[key['code']] = key['city']

airport_city_code['JFK']

'Jamaica'

In [110]:
def get_city_from(airport_code):
    return airport_city_code.get(airport_code, "N/A")

flight_df['ORIGIN_AIRPORT'].head().apply(get_city_from)

0        Anchorage
1      Los Angeles
2    San Francisco
3      Los Angeles
4          Seattle
Name: ORIGIN_AIRPORT, dtype: object

In [111]:
flight_df['origin_city'] = flight_df['ORIGIN_AIRPORT'].apply(get_city_from)

In [112]:
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,WEATHER_DELAY,Delayed,Date,Unix Time,origin_lat,origin_lon,origin_city,dest_lat,dest_lon,dest_city
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,,1/1/2015,1420092000,,,Anchorage,,,
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,,1/1/2015,1420092000,,,Los Angeles,,,
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,,1/1/2015,1420092000,,,San Francisco,,,
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,,1/1/2015,1420092000,,,Los Angeles,,,
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,,1/1/2015,1420092000,,,Seattle,,,


In [41]:
for index, row in flight_df.head(29).iterrows():
    try:
        origin_city = citipy.nearest_city(float(row['origin_lat']), float(row['origin_lon']))
        dest_city = citipy.nearest_city(float(row['dest_lat']), float(row['dest_lon']))
        flight_df.set_value(index, 'origin_city', origin_city.city_name)
        flight_df.set_value(index, 'dest_city', dest_city.city_name)
    except:
        print('error at row ' + str(index))

error at row 0
error at row 1
error at row 2
error at row 3
error at row 4
error at row 5
error at row 6
error at row 7
error at row 8
error at row 9
error at row 10
error at row 11
error at row 12
error at row 13
error at row 14
error at row 15
error at row 16
error at row 17
error at row 18
error at row 19
error at row 20
error at row 21
error at row 22
error at row 23
error at row 24
error at row 25
error at row 26
error at row 27
error at row 28


In [42]:
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,WEATHER_DELAY,Delayed,Date,Unix Time,origin_lat,origin_lon,origin_city,dest_lat,dest_lon,dest_city
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,,1/1/2015,1420092000,,,,,,
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,,1/1/2015,1420092000,,,,,,
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,,1/1/2015,1420092000,,,,,,
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,,1/1/2015,1420092000,,,,,,
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,,1/1/2015,1420092000,,,,,,


In [43]:
cities.append(city.city_name)

NameError: name 'cities' is not defined

In [ ]:
weather = req.get(weather_url).json()

In [ ]:
weather